# Analyse de l'ensemble des Dataframe d'un vol

Projet de statistiques descriptives MACS3, dans le cadre du cours de Jérôme Lacaille.

_Version 1.0 [28/11/2023] Kraemer Valentin_

## Analyse de notre problématique d'inclinaison de l'avion en fonction de l'angle du levier - cas multi-vols

Dans un second temps, nous appliquerons ces outils d'analyse dans le cas de notre tentative d'interprétation de l'inclinaison de l'avion par rapport à l'angle du levier.